In [ ]:
import os.path as osp

# computation
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# data pipeline
import imageio
from imgaug import augmenters as iaa
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# utils
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# BELOW CELLS SHOULD BE AS SAME AS YOUR TRAINING NOTEBOOK

In [ ]:
# Some constants
ROOT = '/kaggle/input/cassava-leaf-disease-classification'
TRAIN_DIR = f'{ROOT}/train_images/'
TRAIN_CSV = f'{ROOT}/train.csv'
TEST_DIR = f'{ROOT}/test_images/'
TEST_CSV = f'{ROOT}/sample_submission.csv'

In [ ]:
# WARNING: THIS PART SHOULD BE AS SAME AS YOUR TRAINING NOTEBOOK!
class CassavaDataset(Dataset):
    def __init__(self, split, transform=None):
        assert split in ('train', 'val', 'test')
        self.split = split
        self.transform = transform
        if split in ('train', 'val'):
            csv = pd.read_csv(TRAIN_CSV)
            self.df = train_test_split(
                csv, test_size=0.1, random_state=0
            )[0 if split == 'train' else 1].reset_index()
        else:
            self.df = pd.read_csv(TEST_CSV)
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, i: int):
        base_dir = TRAIN_DIR if self.split in ('train', 'val') else TEST_DIR
        x = imageio.imread(osp.join(base_dir, self.df['image_id'][i]))
        y = self.df['label'][i] if self.split in ('train', 'val') else -1
        if self.transform:
            x = self.transform(x)
        return (x, y)

In [ ]:
# WARNING: THIS PART SHOULD BE AS SAME AS YOUR TRAINING NOTEBOOK!
INPUT_SIZE = 128
TRANSFORMS = {
    'train': transforms.Compose([
        iaa.Sequential([
            iaa.Resize((INPUT_SIZE, INPUT_SIZE)),
        ]).augment_image,
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        iaa.Sequential([
            iaa.Resize((INPUT_SIZE, INPUT_SIZE)),
        ]).augment_image,
        transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
        iaa.Sequential([
            iaa.Resize((INPUT_SIZE, INPUT_SIZE)),
        ]).augment_image,
        transforms.ToTensor(),
    ]),
}

In [ ]:
# WARNING: THIS PART SHOULD BE AS SAME AS YOUR TRAINING NOTEBOOK!
class CassavaClassifier(nn.Module):
    def __init__(self):

        super().__init__()
        CH_LAST = 128
        NUM_CLASSES = 5
        # TODO: Change this part to improve your score
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, CH_LAST, kernel_size=3, padding=1)
        # NOTE: input channel should match CH_LAST * (IMAGE_INPUT_SIZE / 2 ** NUM_DOWNSAMPLE) ** 2 
        self.fc = nn.Linear(CH_LAST *  (INPUT_SIZE // 2 ** 3) ** 2, NUM_CLASSES)
        self.pool = nn.MaxPool2d(kernel_size=(3, 3), stride=2, padding=1)
        
    def forward(self, x):
        h = self.pool(F.relu(self.conv1(x)))
        h = self.pool(F.relu(self.conv2(h)))
        h = self.pool(F.relu(self.conv3(h)))
        return self.fc(h.flatten(1))

# EVALUATION

1. Turn off the GPU acceleration in your training notebook.
2. **Turn on** the GPU acceleration for this notebook, and **turn off the internet** in "Settings" at the right-side panel.
3. Click the **"Add data"** button at the right-side panel.
4. Click **"Notebook Output Files"** - **"Your Work"** and find your *training* notebook that has been saved with **"Save and Run All"**.
5. Run this notebook to verify that the trained model can be loaded without any errors.
6. Save this notebook with **"Save and Run All"**.
7. Submit the saved version of this notebook & output files to the challenge!

In [ ]:
# WARNING: MAKE SURE THIS PATH MATCHES YOUR IMPORTED OUTPUTS FILES OF THE TRAINING NOTEBOOK 
PATH = '../input/cassava-classification-lg-cv-2021-train/model.pt'
model = torch.load(PATH)
model.eval()
test_dataset = CassavaDataset('test', transform=TRANSFORMS['test'])
test_loader = DataLoader(test_dataset, batch_size=32)
test_csv = pd.read_csv(TEST_CSV)

y_hats = []
for x, _ in test_loader:
    y_hat = model(x.cuda())
    y_hat = torch.argmax(y_hat,dim=1)
    y_hats.extend(y_hat.cpu().detach().numpy().tolist())
    

test_csv['label'] = y_hats
test_csv[['image_id','label']].to_csv("submission.csv", index=False)
test_csv.head()